In [1]:
import ee
import logging
import multiprocessing
import requests
import shutil
from retry import retry

In [2]:
"""
This tool demonstrates extracting data from Earth Engine using parallel
request and getThumbURL.
"""

ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [3]:
def getRequests():
  """Generates a list of work items to be downloaded.

  Produces 1000 random points in each of the RESOLVE ecoregions in the ROI.
  """
  # Get our ROI from the GAUL regions
  gaul = ee.FeatureCollection('FAO/GAUL/2015/level1')
  roi = gaul.filter('ADM1_NAME == "Colorado"').first().geometry()

  # To stratify by RESOLVE ecoregion, paint the ECO_IDs into an image.
  resolve = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017')
  ecoregions = (resolve.reduceToImage(['ECO_ID'], ee.Reducer.first())
                .clip(roi).rename('ECO_ID'))
  points = ecoregions.stratifiedSample(
      numPoints=10,
      classBand='ECO_ID',
      region=roi,
      scale=100,
      geometries=True)

  return points.aggregate_array('.geo').getInfo()


@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
  """Handle the HTTP requests to download an image."""

  # Generate the desired image from the given point.
  point = ee.Geometry.Point(point['coordinates'])
  region = point.buffer(127).bounds()
  image = (ee.ImageCollection('USDA/NAIP/DOQQ')
           .filterBounds(region)
           .filterDate('2019', '2020')
           .mosaic()
           .clip(region)
           .select('R', 'G', 'B'))

  # Fetch the URL from which to download the image.
  url = image.getThumbURL({
      'region': region,
      'dimensions': '256x256',
      'format': 'png'})

  # Handle downloading the actual pixels.
  r = requests.get(url, stream=True)
  if r.status_code != 200:
    r.raise_for_status()

  filename = 'tile_%05d.png' % index
  with open(filename, 'wb') as out_file:
    shutil.copyfileobj(r.raw, out_file)
  print("Done: ", index)

In [ ]:
if __name__ == '__main__':
  logging.basicConfig()
  items = getRequests()

  pool = multiprocessing.Pool(25)
  pool.starmap(getResult, enumerate(items))

  pool.close()

In [ ]:
getRequests()